In [23]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Json"
  // #r "nuget: Plotly.NET.ImageExport"

open Plotly.NET
// open Plotly.NET.ImageExport  

#load "Helpers.fs"
#load "Bestiary.fs"
#load "Compare.fs"
#load "DamageDistribution.fs"
#load "Library.fs"
#load "Transform.fs"

open PathfinderAnalysis.Library
open PathfinderAnalysis.Bestiary
open PathfinderAnalysis.Compare
open PathfinderAnalysis.Transform
open PathfinderAnalysis.Helpers

Installed Packages FSharp.Json, 0.4.1 Plotly.NET.Interactive, 5.0.0

In [ ]:
let titleFn level = 
  sprintf "Middle Save - Level %i - Spout Cantrip vs. Martial Bow Against Level %i Creatures" level (level+2)

let mapShortbowAverages =
  transformedResultsByRollByLevel martialShortbow PlayerAttack creatureAc ((+) -5 << highMartialAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

let shortbowAverages =
  transformedResultsByRollByLevel martialShortbow PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel spout CreatureSave middleSave (casterDc true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Spout"
  };
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel telekineticProjectile PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Telekinetic Projectile"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel martialArbalest PlayerAttack creatureAc (highMartialAttack true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.toArray;
    Title = "Arbalest (1 shot)"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel shortbowAverages mapShortbowAverages
      |> Seq.toArray;
    Title = "Shortbow (2 shots)"
  }
|]
|> flatten
|> generateCharts titleFn
// |> Seq.iteri (fun i chart -> Chart.savePNG (path = (sprintf "spout-arbalest-%02i" i)) chart)
// |> ignore


In [21]:
let titleFn level = 
  sprintf "Middle Save - Level %i - Caster Focus Spells vs. Fighter Weapons Against Level %i Creatures" level (level+2)

let mapShortbowAverages =
  transformedResultsByRollByLevel fighterLongbow PlayerAttack creatureAc ((+) -5 << highFighterAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

let shortbowAverages =
  transformedResultsByRollByLevel fighterLongbow PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

let doubleShotAverages =
  transformedResultsByRollByLevel fighterShortbow PlayerAttack creatureAc ((+) -2 << highFighterAttack true) bestiaryByLevel 2
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel tempestSurge CreatureSave middleSave (casterDc true) bestiaryByLevel 2
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Tempest Surge"
  };
  // { 
  //   AveragesByRollsByLevel = transformedResultsByRollByLevel fireRay PlayerAttack creatureAc (casterAttack true false) bestiaryByLevel 2
  //     |> Seq.map resultRollsToAverages
  //     |> Seq.toArray;
  //   Title = "Fire Ray";
  // };
  // {
  //   AveragesByRollsByLevel = transformedResultsByRollByLevel fighterArbalest PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel 2
  //     |> Seq.map resultRollsToAverages
  //     |> Seq.toArray;
  //   Title = "Arbalest (1 shot)"
  // };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel shortbowAverages mapShortbowAverages
      |> Seq.toArray;
    Title = "Longbow (2 shots)"
  };
  // {
  //   AveragesByRollsByLevel = mergeRollAveragesByLevel doubleShotAverages doubleShotAverages
  //     |> Seq.toArray;
  //   Title = "Shortbow (Double Shot)"
  // }
|]
|> flatten
|> generateCharts titleFn

[ MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions) ... (more) ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Tuple`2[System.Int32,System.Collections.Generic.IEnumerable`1[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]],Plotly.NET.GenericChart] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Tuple`2[System.Int32,System.Collections.Generic.IEnumerable`1[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]],Plotly.NET.GenericChart] mapping FSI_0181.PathfinderAnalysis.Transform+generateCharts@88-18 titleFn FSI_0190+it@48-64 ie [ (1, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (2, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (3, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (4, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (5, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (6, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (7, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (8, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (9, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0181.PathfinderAnalysis.Transform+FlatChartData]), (10, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI

In [ ]:
let monsterLevelDiff = 4

let titleFn level = 
  sprintf "Middle Save - Level %i - Caster Single-Target vs. Fighter Weapons Against Level %i Creatures" level (level+monsterLevelDiff)

let longswordAverage =
  transformedResultsByRollByLevel fighterLongsword PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
  |> Seq.map resultRollsToAverages

let shortswordAverage =
  transformedResultsByRollByLevel fighterShortsword PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
  |> Seq.map resultRollsToAverages

[|
  { 
    AveragesByRollsByLevel = transformedResultsByRollByLevel thunderstrike CreatureSave middleSave (casterDc true) bestiaryByLevel monsterLevelDiff
      |> Seq.map resultRollsToAverages
      |> Seq.map normalizeSavingThrowsForLevel
      |> Seq.toArray;
    Title = "Thunderstrike"
  };
  {
    AveragesByRollsByLevel = mergeRollAveragesByLevel longswordAverage shortswordAverage
      |> Seq.toArray;
    Title = "Double Slice"
  };
  {
    AveragesByRollsByLevel = transformedResultsByRollByLevel (forceBarrage 3) PlayerAttack creatureAc (highFighterAttack true) bestiaryByLevel monsterLevelDiff
    |> Seq.map resultRollsToAverages
    |> Seq.toArray;
    Title = "3-action force barrage"
  }
|]
|> flatten
|> generateCharts titleFn

In [6]:
open PathfinderAnalysis.DamageDistribution

let dicePool = [D12, 4; D6, 3; D4, 2]
let nBuckets = 20

let values = 
  rollDistributions 0 dicePool
  |> Seq.toList
  |> chunkRolls nBuckets
  |> chunksToAverages
  |> Seq.toList

let title = sprintf "Roll Sum of %A mapped onto a d%i" dicePool nBuckets

Chart.Line(
  x = List.map first values,
  y = List.map second values,
  ShowMarkers = true,
  MultiText = (List.map second values |> List.map (sprintf "%.0f")),
  TextPosition = StyleParam.TextPosition.TopCenter
)
|> Chart.withXAxisStyle(TitleText = "D20 Result")
|> Chart.withYAxisStyle(TitleText = "Estimated Roll Sum")
|> Chart.withTitle(title)



<!-- Plotly chart will be drawn inside this DIV -->

In [28]:
open PathfinderAnalysis.DamageDistribution

let creatureLevelBump = 2

let titleFn level =
  sprintf "Middle Save - Level %i - Druid vs. Fighter Against Level %i Creatures" level (level + creatureLevel - pcLevel)

let fighterLongbowBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongbow pcLevel };
    { HitRollCount = 1; HitModifier = highFighterAttack true pcLevel - 5; Contest = PlayerAttack; CreatureDefenseFunction = creatureAc; DamageFunction = diceFighterLongbow pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

let druidTempestSurgeBucketsForLevel pcLevel =
  [|
    { HitRollCount = 1; HitModifier = casterDc true pcLevel; Contest = CreatureSave; CreatureDefenseFunction = middleSave; DamageFunction = diceDamageTempestSurge pcLevel };
  |]
  |> bigThingStandardChunk bestiaryByLevel[pcLevel + creatureLevelBump]

[1..20]
|> List.map (fun level -> ["Tempest Surge", level, druidTempestSurgeBucketsForLevel level; "Longbow", level, fighterLongbowBucketsForLevel level])
|> Seq.collect (fun titledBucketsByLevel -> Seq.map (fun (title, level, bucket) -> chunksToFlatChartData title level bucket) titledBucketsByLevel)
|> generateCharts titleFn


[ MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions), MultiChart
 ([Plotly.NET.Trace2D; Plotly.NET.Trace2D], Plotly.NET.Layout,
 Plotly.NET.Config, Plotly.NET.DisplayOptions) ... (more) ] f Microsoft.FSharp.Collections.SeqModule+revamp@728[System.Tuple`2[System.Int32,System.Collections.Generic.IEnumerable`1[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]],Plotly.NET.GenericChart] f Microsoft.FSharp.Collections.SeqModule+Map@748-1[System.Tuple`2[System.Int32,System.Collections.Generic.IEnumerable`1[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]],Plotly.NET.GenericChart] mapping FSI_0201.PathfinderAnalysis.Transform+generateCharts@88-24 titleFn FSI_0220+it@24-92 ie [ (1, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (2, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (3, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (4, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (5, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (6, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (7, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (8, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (9, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI_0201.PathfinderAnalysis.Transform+FlatChartData]), (10, Microsoft.FSharp.Collections.IEnumerator+mkSeq@177[FSI